## L-mode H-mode Classifier
- In this notebook a single camera model will be created and trained. 
- First import ResNet model, modify its fc. layer and train this fc.layer
- Then train the whole model
- We can either train the model on imgs from RIS1 camera or RIS2, and then ensemble these models in `ModelEnsembling.ipynb`

In [1]:
import os
from pathlib import Path
import matplotlib.pyplot as plt
import confinement_mode_classifier as cmc
from datetime import datetime
import time 
import re
import torchvision
import torch
from torch.optim import lr_scheduler
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter

Seed set to 42


Device: cuda:0


In [2]:
path = Path(os.getcwd())
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

#Where are csv spreadsheets with time, mode, img_path, h_alpha columns are saved
data_dir_path = f'{path}/data/LH_alpha'
file_names = os.listdir(data_dir_path)


Choose shots, which will be used in training

In [3]:
shot_numbers = [re.search(r'shot_(\d+)', file_name).group(1) for file_name in file_names]
removed_shots = ['19915', '19925', '13182', '20009', '20112', 
                 '20143', '20145', '20146', '20147', '16989', 
                 '16987', '20144', '18263', '18267', '18266', 
                 '18279', '20098', '18260', '18200', '18268', '18261']
shot_numbers = [valid_shot for valid_shot in shot_numbers if valid_shot not in removed_shots]

shots_for_testing = ['18130', '16773', '16534', '19094', '18133']
shots_for_validation = ['16769', '19379', '18057', '18132']

### Create dataloader
- It is convinient to have dloaders in dictionary in order to easily switch between training and validation

In [4]:
batch_size = 32
shot_df, test_df, val_df, train_df = cmc.load_and_split_dataframes(path,shot_numbers, shots_for_testing, 
                                                                   shots_for_validation, use_ELMS=False)


test_dataloader = cmc.get_dloader(test_df, path, batch_size, ris_option='RIS2', balance_data=True, shuffle=False)
val_dataloader = cmc.get_dloader(val_df, path, batch_size, ris_option='RIS2', balance_data=True, shuffle=False)
train_dataloader = cmc.get_dloader(train_df, path, batch_size, ris_option='RIS2', balance_data=True, shuffle=False)

dataloaders = {'train':train_dataloader, 'val':val_dataloader}
dataset_sizes = {x: len(dataloaders[x].dataset) for x in ['train', 'val']}

/compass/Shared/Users/bogdanov/.venv/lib/python3.8/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/compass/Shared/Users/bogdanov/.venv/lib/python3.8/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Time stamp will be added to the model's folder name

In [5]:
timestamp =  datetime.fromtimestamp(time.time()).strftime("%d-%m-%y, %H-%M-%S ") + input('add comment: ')
writer = SummaryWriter(f'runs/{timestamp}_last_fc')

### Import ResNet pretrained model
 And freeze all layers except last f.c. layer


In [6]:
pretrained_model = torchvision.models.resnet18(weights='IMAGENET1K_V1', )
for param in pretrained_model.parameters():
    param.requires_grad = False
 
# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = pretrained_model.fc.in_features
pretrained_model.fc = nn.Linear(num_ftrs, 3) #3 classes: L-mode, H-mode, ELM
pretrained_model = pretrained_model.to(device)

### Train last fc of the model

In [ ]:
#
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = torch.optim.Adam(pretrained_model.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.OneCycleLR(optimizer, max_lr=1e-2) #!!!

#Model will be saved to this folder along with metrics and tensorboard scalars
model_path = Path(f'{path}/runs/{timestamp}_last_fc/model.pt')

num_epochs = 10
model = cmc.train_model(pretrained_model, criterion, optimizer, exp_lr_scheduler, 
                       dataloaders, writer, dataset_sizes, num_epochs=num_epochs, 
                       chkpt_path=model_path.with_name(f'{model_path.stem}_chkpt{model_path.suffix}'))

torch.save(model.state_dict(), model_path)

### Train the whole model

In [ ]:
writer = SummaryWriter(f'runs/{timestamp}_all_layers')
for param in model.parameters():
    param.requires_grad = True

model_path = Path(f'{path}/runs/{timestamp}/model.pt')


model = cmc.train_model(pretrained_model, criterion, optimizer, exp_lr_scheduler, 
                        dataloaders, writer, dataset_sizes, num_epochs=14,
                        chkpt_path=model_path.with_name(f'{model_path.stem}_chkpt{model_path.suffix}'))
torch.save(model.state_dict(), model_path)

### Test model with all layers trained

In [9]:

#metrics = cmc.test_model(f'runs/{timestamp}', model, test_dataloader, comment='all_layers')

# writer.add_figure(f'Confusion matrix for the model with trained f.c. layer', fig_confusion_matrix)
# writer.add_scalar(f'Accuracy on test_dataset with trained f.c. layer', accuracy)
# writer.add_scalar(f'F1 metric on test_dataset with trained f.c. layer', f1)
# writer.add_scalar(f'Precision on test_dataset with trained f.c. layer', precision)
# writer.add_scalar(f'Recall on test_dataset with trained f.c. layer', recall)
# writer.close()